In [ ]:
import nltk
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
from textblob import TextBlob
import requests
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('stopwords')
nltk.download('words')
stop_words = stopwords.words('english')
words = set(nltk.corpus.words.words())
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
  def load_data_label(path):
      temp_data = []
      lines = [line.strip() for line in open(path)]
      for idx in range(0, len(lines), 4):
          id = lines[idx].split("\t")[0]
          relation = lines[idx + 1]

          sentence = lines[idx].split("\t")[1][1:-1]
          sentence = sentence.replace("<e1>", " _e1_ ").replace("</e1>", " _/e1_ ")
          sentence = sentence.replace("<e2>", " _e2_ ").replace("</e2>", " _/e2_ ")
          sentence = sentence.replace("<e1>", "<e1> ").replace("</e1>", " </e11>")
          sentence = sentence.replace("<e2>", "<e2> ").replace("</e2>", " </e22>")

          tokens = nltk.word_tokenize(sentence)

          tokens.remove('_/e1_')
          tokens.remove('_/e2_')

          e1 = tokens.index("_e1_")
          del tokens[e1]
          element1=tokens[e1]
          e2 = tokens.index("_e2_")
          del tokens[e2]
          element2=tokens[e2]
          sentence = " ".join(tokens)
          temp_data.append([id, sentence, e1, element1, e2, element2, relation])
      df = pd.DataFrame(data=temp_data, columns=["id", "sentence", "e1_position","element1", "e2_position","element2", "class"])
      #print (df)
      labelsMapping = {'Other': 0,'Message-Topic(e1,e2)': 1, 'Message-Topic(e2,e1)': 2,
                      'Product-Producer(e1,e2)': 3, 'Product-Producer(e2,e1)': 4,
                      'Instrument-Agency(e1,e2)': 5, 'Instrument-Agency(e2,e1)': 6,
                      'Entity-Destination(e1,e2)': 7, 'Entity-Destination(e2,e1)': 8,
                      'Cause-Effect(e1,e2)': 9, 'Cause-Effect(e2,e1)': 10,
                      'Component-Whole(e1,e2)': 11, 'Component-Whole(e2,e1)': 12,
                      'Entity-Origin(e1,e2)': 13, 'Entity-Origin(e2,e1)': 14,
                      'Member-Collection(e1,e2)': 15, 'Member-Collection(e2,e1)': 16,
                      'Content-Container(e1,e2)': 17, 'Content-Container(e2,e1)': 18}
      df['tag'] = [labelsMapping[r] for r in df['class']]
      #print(df)
      x_sentence = df['sentence'].tolist()

      #Label Data
      y = df['tag']
      return df

In [ ]:
df = pd.read_csv("large_ds_df_exported.csv")
path_to_test_file = "/content/TEST_FILE.txt"
df_test = load_data_label(path_to_test_file)

In [ ]:
def preprocessor(sentence):

    sentence = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", sentence)
    sentence = sentence.lower()
    sentence = re.sub(r'\d+','', sentence)
    sentence = sentence.replace("user", "")
    return  sentence

In [ ]:
def clean_text(df):

    train_cleaned = df['sentence'].apply(preprocessor)
    df['sentence'] = train_cleaned.apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

    return df

In [ ]:
#cleaning the text
train_cleaned = clean_text(df)
test_cleaned = clean_text(df_test)

In [ ]:
import pandas as pd

def insert_special_tokens(df):
    """
    Inserts special tokens ($ and #) around entity1 and entity2 in each sentence.
    Adjusts spacing and punctuation accordingly.
    Ensures consistency by removing trailing full stops.
    """
    modified_sentences = []

    for _, row in df.iterrows():
        sentence = row['sentence']
        e1 = row['element1']
        e2 = row['element2']

        # Ensure correct token placement
        sentence = sentence.replace(e1, f"$ {e1} $").replace(e2, f"# {e2} #")

        # Handle punctuation spacing
        sentence = sentence.replace(" .", ".").replace(" ,", ",").replace(" '", "'")

        # Remove trailing full stop if present
        if sentence.endswith(" ."):
            sentence = sentence[:-2]  # Remove space + period
        elif sentence.endswith("."):
            sentence = sentence[:-1]  # Remove period

        modified_sentences.append(sentence)

    df['sentence'] = modified_sentences
    return df

# Apply transformation
new_train_cleaned = train_cleaned.copy()
new_test_cleaned = test_cleaned.copy()
new_train_cleaned = insert_special_tokens(new_train_cleaned)
new_test_cleaned = insert_special_tokens(new_test_cleaned)

with pd.option_context('display.max_colwidth', None):
  print(new_train_cleaned['sentence'].head())
  print(new_test_cleaned['sentence'].head())

0                      the system described above has its greatest application arrayed $ configuration $ antenna # elements #
1                                                the $ child $ was carefully wrapped and bound into the # cradle # means cord
2                                                      the $ author $ keygen uses # disassembler # look the raw assembly code
3                                                                                  misty $ ridge $ uprises from the # surge #
4    the $ student $ # association # the voice the undergraduate $ student $ population the state university new york buffalo
Name: sentence, dtype: object
0                                   the most common $ audits $ were about # waste # and recycling
1                                                   the $ company $ fabricates plastic # chairs #
2                                         the school $ master $ teaches the lesson with # stick #
3                                   the suspec

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
pip install transformers

In [ ]:
from transformers import BertTokenizer

# loading BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Add new special tokens
special_tokens_dict = {'additional_special_tokens': ["$", "#"]}
tokenizer.add_special_tokens(special_tokens_dict)

#  tokenizing set of texts
def Input_embeddings(data,length):

    input_ids = []
    attention_masks = []

    for sentence in data:

        encoded_sent = tokenizer.encode_plus(
            text=sentence,
            add_special_tokens=True,
            max_length=length,
            pad_to_max_length=True,
            return_attention_mask=True
            )
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
sentences = np.concatenate([new_train_cleaned.sentence.values, new_test_cleaned.sentence.values])
sentences_encoded = [tokenizer.encode(sent, add_special_tokens=True, truncation=True) for sent in sentences]
max_length = max([len(sent) for sent in sentences_encoded])
print('Max length: ', max_length)

Max length:  139


In [ ]:
data = [new_train_cleaned.sentence[0]]
token_ids = list(Input_embeddings(data,max_length)[0].squeeze().numpy())
print('Original: ', train_cleaned.sentence[0])
print('Token IDs: ', token_ids)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Original:  the system described above has its greatest application arrayed configuration antenna elements
Token IDs:  [101, 1996, 2291, 2649, 2682, 2038, 2049, 4602, 4646, 9140, 2098, 1002, 9563, 1002, 13438, 1001, 3787, 1001, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import torch.nn as nn
import torch.nn.functional as F
import time
import datetime

# Split the data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    new_train_cleaned['sentence'].values,
    new_train_cleaned['tag'].values,
    test_size=0.1,
    random_state=42
)

# Tokenize all of the sentences and map the tokens to their word IDs.
train_input_ids, train_attention_masks = Input_embeddings(train_sentences, max_length)
val_input_ids, val_attention_masks = Input_embeddings(val_sentences, max_length)

# Convert the labels to tensors.
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create the DataLoader for our training set.
train_data = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

# Create the DataLoader for our validation set.
val_data = TensorDataset(val_input_ids, val_attention_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)



In [ ]:
from collections import Counter
import torch

# Calculate class counts
class_counts = Counter(train_cleaned['tag'])

# Exclude class 8 from the class counts
class_counts_without_8 = {k: v for k, v in class_counts.items() if k != 8}

# Calculate class weights (inverse of class frequencies) excluding class 8
class_weights_without_8 = torch.tensor([1.0 / class_counts_without_8[i] for i in range(19) if i != 8], dtype=torch.float32)

# Normalize the weights so they sum to 1
class_weights_without_8 = class_weights_without_8 / class_weights_without_8.sum()

# Create a tensor for all classes, initially setting weight for class 8 to 0
class_weights = torch.zeros(19, dtype=torch.float32)
for i in range(19):
    if i != 8:
        class_weights[i] = class_weights_without_8[i if i < 8 else i - 1]

#manual adjust
class_weights[8]=0.02
class_weights[14]=0.0786
class_weights[18]=0.0916
print(class_weights)
print(class_weights.sum())

tensor([0.0079, 0.0229, 0.0822, 0.0351, 0.0263, 0.1742, 0.0397, 0.0106, 0.0200,
        0.0602, 0.0268, 0.0268, 0.0252, 0.0198, 0.0786, 0.1840, 0.0265, 0.0415,
        0.0916])
tensor(1.0000)


In [ ]:
class_weights = class_weights.to(device)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

In [ ]:


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Function to calculate the F1 score
from sklearn.metrics import f1_score

def flat_f1(preds, labels):
    # Flatten the predictions and labels
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    # Exclude the 'Other' class (label 0) and calculate F1 for the remaining 18 classes
    f1 = f1_score(labels_flat, pred_flat, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], average='macro')
    return f1

In [ ]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",  # Use the 12-layer BERT model, with an uncased vocab.
    num_labels=19,  # The number of output labels--19 for our task.
    output_attentions=False,  # Whether the model returns attentions weights.
    output_hidden_states=False,  # Whether the model returns all hidden-states.
)

model.resize_token_embeddings(len(tokenizer))
# Move the model to the device (GPU or CPU)
model.to(device)

# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
optimizer = AdamW(
    model.parameters(),
    lr=2e-5,  # args.learning_rate - default is 5e-5, our notebook had 2e-5
    eps=1e-8,  # args.adam_epsilon  - default is 1e-8.
)

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,  # Default value in run_glue.py
    num_training_steps=total_steps,
)
# Training loop
import random
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    model.train()
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        # Move batch tensors to the device
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits  # Extract logits from the model output

        # Calculate loss using the custom loss function with class weights
        loss = loss_fn(logits, b_labels)
        total_loss += loss.item()

        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================
    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()
    eval_accuracy, eval_f1 = 0, 0

    for batch in val_dataloader:
        # Move batch tensors to the device
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            # Forward pass
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            logits = outputs.logits

        # Move logits and labels to CPU for metric calculation
        logits = logits.detach().cpu().numpy()
        labels = b_labels.to('cpu').numpy()

        # Calculate accuracy and F1 score
        eval_accuracy += flat_accuracy(logits, labels)
        eval_f1 += flat_f1(logits, labels)

    print(f"Validation Accuracy: {eval_accuracy / len(val_dataloader)}")
    print(f"Validation F1 Score: {eval_f1 / len(val_dataloader)}")
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 2.18
  Training epoch took: 0:05:29

Running Validation...
Validation Accuracy: 0.5005549389567148
Validation F1 Score: 0.35654806221306395
  Validation took: 0:00:13

======== Epoch 2 / 4 ========
Training...

  Average training loss: 1.09
  Training epoch took: 0:05:34

Running Validation...
Validation Accuracy: 0.7008185349611542
Validation F1 Score: 0.4839874147310329
  Validation took: 0:00:13

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.64
  Training epoch took: 0:05:34

Running Validation...
Validation Accuracy: 0.7625901775804661
Validation F1 Score: 0.5175046962244911
  Validation took: 0:00:13

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.46
  Training epoch took: 0:05:34

Running Validation...
Validation Accuracy: 0.7855160932297447
Validation F1 Score: 0.5284730788661606
  Validation took: 0:00:13

Training complete!


WITH SELECTED PARAMETERS

In [ ]:
epochs = 5
batch_size = 64
# Update the DataLoader with the new batch size
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)
# Reinitialize the model, optimizer, and scheduler with the new hyperparameters
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=19,
    output_attentions=False,
    output_hidden_states=False,
)
model.to(device)

optimizer = AdamW(
    model.parameters(),
    lr=7.6e-05,
    eps=1e-8,  # Fixed epsilon
    weight_decay=0.01,  # Tuned weight decay
)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),  # 10% warmup steps
    num_training_steps=total_steps,
)
# Training loop
import random
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    model.train()
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        # Move batch tensors to the device
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits  # Extract logits from the model output

        # Calculate loss using the custom loss function with class weights
        loss = loss_fn(logits, b_labels)
        total_loss += loss.item()

        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================
    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()
    eval_accuracy, eval_f1 = 0, 0

    for batch in val_dataloader:
        # Move batch tensors to the device
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            # Forward pass
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            logits = outputs.logits

        # Move logits and labels to CPU for metric calculation
        logits = logits.detach().cpu().numpy()
        labels = b_labels.to('cpu').numpy()

        # Calculate accuracy and F1 score
        eval_accuracy += flat_accuracy(logits, labels)
        eval_f1 += flat_f1(logits, labels)

    print(f"Validation Accuracy: {eval_accuracy / len(val_dataloader)}")
    print(f"Validation F1 Score: {eval_f1 / len(val_dataloader)}")
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



======== Epoch 1 / 5 ========
Training...

  Average training loss: 2.15
  Training epoch took: 0:05:32

Running Validation...
Validation Accuracy: 0.6023624727668846
Validation F1 Score: 0.518863111971555
  Validation took: 0:00:13

======== Epoch 2 / 5 ========
Training...

  Average training loss: 0.70
  Training epoch took: 0:05:31

Running Validation...
Validation Accuracy: 0.8105936819172114
Validation F1 Score: 0.6685084501788436
  Validation took: 0:00:13

======== Epoch 3 / 5 ========
Training...

  Average training loss: 0.28
  Training epoch took: 0:05:31

Running Validation...
Validation Accuracy: 0.8583537581699346
Validation F1 Score: 0.6925881080769982
  Validation took: 0:00:13

======== Epoch 4 / 5 ========
Training...

  Average training loss: 0.12
  Training epoch took: 0:05:31

Running Validation...
Validation Accuracy: 0.8843954248366013
Validation F1 Score: 0.7044196490645037
  Validation took: 0:00:13

======== Epoch 5 / 5 ========
Training...

  Average trainin

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

# Tokenize the test sentences and prepare the test DataLoader
test_input_ids, test_attention_masks = Input_embeddings(new_test_cleaned['sentence'].values, max_length)
test_labels = torch.tensor(new_test_cleaned['tag'].values)

# Create the TensorDataset
test_data = TensorDataset(test_input_ids, test_attention_masks, test_labels)

# Create the DataLoader
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=32)

In [ ]:
# Test evaluation loop
model.eval()
test_accuracy, test_f1 = 0, 0
nb_test_steps = 0

for batch in test_dataloader:
    # Move batch to GPU
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():
        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]

    # Calculate predictions on GPU
    preds = torch.argmax(logits, dim=1)

    # Calculate accuracy on GPU
    correct_predictions = torch.sum(preds == b_labels).item()
    total_predictions = len(b_labels)
    tmp_test_accuracy = correct_predictions / total_predictions

    # Calculate F1 score on GPU (move to CPU temporarily for sklearn)
    tmp_test_f1 = f1_score(
        b_labels.cpu().numpy(),  # Move labels to CPU for sklearn
        preds.cpu().numpy(),     # Move predictions to CPU for sklearn
        average='macro',         # Macro-averaged F1 score
        labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]  # Exclude 'Other' class
    )

    # Accumulate accuracy and F1 score
    test_accuracy += tmp_test_accuracy
    test_f1 += tmp_test_f1
    nb_test_steps += 1

# Report final test metrics
print(f"Test Accuracy: {test_accuracy / nb_test_steps}")
print(f"Test F1 Score (excluding 'Other' class): {test_f1 / nb_test_steps}")

Test Accuracy: 0.7982251521298174
Test F1 Score (excluding 'Other' class): 0.5556076603135428
